In [1]:
import os
import sys
import pandas as pd
sys.path.append(os.path.dirname(os.path.abspath(os.getcwd())) + "/..")
if os.getcwd().endswith("Topic1"):
    os.chdir("..")
    print(f"Changed to {os.getcwd()}")
from codebase import get_total_inhabitants_and_willingness, number_of_residents_in_detour

Changed to c:\Users\20203721\OneDrive - TU Eindhoven\Documents\1BM130 Design of AI driven business operations\Design-of-AI-driven-business-operation


In [2]:
within_mins = 50
punt = "trein" # ibis, hbo_wo, mbo, basis, highschool, bus, trein
mode = "fiets" # fiets, ebike
location = "Train - NS" # Elementary Schools, High Schools, Universities, Train - NS, Train - Local

total, total_willing = get_total_inhabitants_and_willingness(punt1=punt, mode=mode, within_mins=within_mins, location=location, verbose=True)

Loaded file from CSV
Total inhabitants within 50 minutes of trein from buurt: 16790525 of 17811291 = 94.27% of the Netherlands
Total willingness to cycle of those: 9752691 of 16790525 = 58.08%


In [3]:
detour_factor = 1.3
within_mins = 50

number_of_residents_in_detour(detour_factor, punt, mode, within_mins)

Loaded file from CSV


,train_id,bu_code,reistijd_min,reisafstand_m,afstand_hemelsbreed_m,omrijdfactor,a_inw
4411,473,BU05991699,36.0,10702.0,7810,1.37,23180
2561,461,BU05990660,10.0,2500.0,1812,1.38,16930
3166,232,BU06370601,14.0,3894.0,2852,1.37,15885
615,466,BU05991284,3.0,847.0,647,1.31,14725
334,415,BU02680970,3.0,929.0,402,2.31,13400
...,...,...,...,...,...,...,...
4034,177,BU19040305,26.0,7143.0,4418,1.62,0
1341,525,BU07180702,5.0,1495.0,1096,1.36,0
150,572,BU06420204,2.0,592.0,287,2.06,0
1170,281,BU03881001,5.0,1459.0,863,1.69,0
